In [1]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
import matplotlib.pyplot as plt
from scipy import stats, signal
from numpy import save, load
import pandas as pd
import sklearn
from sklearn.svm import SVC

### read dataset X and Y

In [2]:
subject_id=1
base_path = "features/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
# files=len(edf_file_names)
X=load('features/data_chb01/features_{}_00.npy'.format(subject_id))
y=load('features/data_chb01/targets_{}_00.npy'.format(subject_id))
for subject_id in range(1,2):
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    print(files)
    print(X.shape)
    start=0
    if subject_id==1:
        start=1
    for fileno in range(start, files//2):
        print(fileno)
        X=np.concatenate((X, load('features/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
        y=np.concatenate((y, load('features/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))


84
(408, 208)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [4]:
X_shape, y_shape = X.shape, y.shape
print(X_shape, y_shape)

(14584, 208) (14584,)


In [9]:
df=pd.DataFrame(data=X)
df['target']=y
print(df.shape)
df=df.dropna()
print(df.shape)

(14584, 209)
(14584, 209)


### train test split

In [10]:
df_interictal=df[df['target']==0]
# print(df_interictal.shape)
# df_interictal=df_interictal.sample(frac=0.7)
df_preictal=df[df['target']==2]
interictal_shape, preictal_shape = df_interictal.shape, df_preictal.shape
print(df_interictal.shape, df_preictal.shape)

(12188, 209) (975, 209)


In [11]:
X_interictal=np.array(df_interictal[df.columns[:-1]]).astype('float32')
y_interictal=np.array(df_interictal['target']).astype('float32')
X_preictal  =np.array(df_preictal[df.columns[:-1]]).astype('float32')
y_preictal  =np.array(df_preictal['target']).astype('float32')

In [12]:
print(X_interictal.shape)
print(y_interictal.shape)
print(X_preictal.shape)
print(y_preictal.shape)

(12188, 208)
(12188,)
(975, 208)
(975,)


### train test split

In [13]:
from sklearn.model_selection import train_test_split


In [14]:
X_interictal_train, X_interictal_test, y_interictal_train, y_interictal_test =train_test_split(X_interictal,y_interictal,test_size=0.1, random_state=42)
X_preictal_train, X_preictal_test, y_preictal_train, y_preictal_test=train_test_split(X_preictal, y_preictal,test_size=0.1, random_state=42)

In [15]:
X_train = np.concatenate((X_interictal_train, X_preictal_train))
X_test = np.concatenate((X_interictal_test, X_preictal_test))
y_train = np.concatenate((y_interictal_train, y_preictal_train))
y_test = np.concatenate((y_interictal_test, y_preictal_test))

In [16]:
X_train_shape =X_train.shape
X_test_shape = X_test.shape
y_train_shape =y_train.shape 
y_test_shape = y_test.shape
print(X_train_shape)
print(X_test_shape)
print(y_train_shape)
print(y_test_shape)

(11846, 208)
(1317, 208)
(11846,)
(1317,)


### Normalization

In [16]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# scaler.transform(X_train)
# scaler.fit(X_test)
# scaler.transform(X_test)

array([[5.06899767e-02, 2.06566319e-01, 2.44740620e-02, ...,
        3.16168010e-01, 8.29013061e-07, 1.00000000e+00],
       [2.38289069e-02, 2.64799237e-01, 1.16495766e-01, ...,
        1.51608899e-01, 2.98875057e-06, 0.00000000e+00],
       [1.77743211e-02, 1.85866445e-01, 1.43353671e-01, ...,
        1.70185298e-01, 9.69898610e-06, 6.66666627e-01],
       ...,
       [2.15109922e-02, 9.67502445e-02, 1.10558691e-02, ...,
        4.12113339e-01, 1.13175914e-03, 1.00000000e+00],
       [6.81196852e-03, 6.83366656e-02, 3.92035069e-03, ...,
        4.36358184e-01, 4.70467021e-05, 6.66666627e-01],
       [4.16661575e-02, 2.63042718e-01, 1.29583791e-01, ...,
        1.59748152e-01, 5.48839807e-06, 3.33333343e-01]], dtype=float32)

### training the model

In [17]:
from datetime import datetime

In [20]:
cost: 100.0
weight_dict: {0: 1, 2: 10}

In [23]:
before = datetime.now()
before_time =before.strftime("%H:%M:%S")
print(before_time)

clf = SVC(C=100, class_weight={0: 1, 2: 10})
clf.fit(X_train, y_train)
trainAcc = clf.score(X_train, y_train)
testAcc = clf.score(X_test, y_test)
print("**SVM Results:**")
print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")

after = datetime.now()
after_time =after.strftime("%H:%M:%S")
print(after_time)

02:52:32
**SVM Results:**
Training Accuracy: 87%
Testing Accuracy: 88%
02:53:07


In [40]:
import joblib

    # estimator = joblib.load("/my_models/%s.pkl"%dataset_name)
    # print "using trained model"

# print ("saving new model")
# joblib.dump(clf,"my_models/SVM_blanced_chb04.pkl")

saving new model


['my_models/SVM_blanced_chb04.pkl']

In [24]:
# loaded_clf = joblib.load("my_models/SVM_blanced_chb04.pkl")
# y_pred=loaded_clf.predict(X_test)
y_pred=clf.predict(X_test)

In [73]:
loaded_clf.get_params(deep=True)

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': 'balanced',
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [25]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, y_pred).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))

TN:1073, FP:146, FN:1, TP:97


In [26]:
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
print('specificity= {} , sensitivity= {}'.format(specificity, sensitivity))

specificity= 0.8802296964725185 , sensitivity= 0.9897959183673469


In [28]:
print('X size ={}, y size = {}'.format(X_shape, y_shape))
print('interictal size ={}, preictal size = {}'.format(interictal_shape, preictal_shape))
print('train size ={}, test size = {}'.format(X_train_shape, X_test_shape))

print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
print('specificity= {} , sensitivity= {}'.format(specificity, sensitivity))

X size =(14584, 208), y size = (14584,)
interictal size =(12188, 209), preictal size = (975, 209)
train size =(11846, 208), test size = (1317, 208)
Training Accuracy: 87%
Testing Accuracy: 88%
TN:1073, FP:146, FN:1, TP:97
specificity= 0.8802296964725185 , sensitivity= 0.9897959183673469


### grid search

In [24]:
# from numpy import mean
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RepeatedStratifiedKFold

In [29]:
# balance = [{0:10,2:1}, {0:1,2:1}, {0:1,2:10}, {0:1,2:50}, {0:1,2:100}]
# costs=[1.0,10.0,100.0]
# param_grid = dict(C=costs, class_weight=balance)

In [36]:
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
# grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1_weighted')


In [37]:
# grid_result = grid.fit(X_train, y_train)

In [38]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.897876 using {'C': 100.0, 'class_weight': {0: 1, 2: 10}}
0.890373 (0.000291) with: {'C': 1.0, 'class_weight': {0: 10, 2: 1}}
0.890373 (0.000291) with: {'C': 1.0, 'class_weight': {0: 1, 2: 1}}
0.749362 (0.010585) with: {'C': 1.0, 'class_weight': {0: 1, 2: 10}}
0.651175 (0.005219) with: {'C': 1.0, 'class_weight': {0: 1, 2: 50}}
0.631188 (0.009186) with: {'C': 1.0, 'class_weight': {0: 1, 2: 100}}
0.890373 (0.000291) with: {'C': 10.0, 'class_weight': {0: 10, 2: 1}}
0.890373 (0.000291) with: {'C': 10.0, 'class_weight': {0: 1, 2: 1}}
0.855444 (0.006939) with: {'C': 10.0, 'class_weight': {0: 1, 2: 10}}
0.784134 (0.010316) with: {'C': 10.0, 'class_weight': {0: 1, 2: 50}}
0.750769 (0.007781) with: {'C': 10.0, 'class_weight': {0: 1, 2: 100}}
0.890373 (0.000291) with: {'C': 100.0, 'class_weight': {0: 10, 2: 1}}
0.890787 (0.000550) with: {'C': 100.0, 'class_weight': {0: 1, 2: 1}}
0.897876 (0.006056) with: {'C': 100.0, 'class_weight': {0: 1, 2: 10}}
0.849596 (0.008939) with: {'C': 100.0, 'c